In [4]:
!pip install torch torchvision


In [6]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import models, transforms
from PIL import Image
import json

# Load pre-trained ResNet model
resnet = models.resnet152(pretrained=True)
modules = list(resnet.children())[:-1]  # Remove the last fully connected layer
resnet = nn.Sequential(*modules)
resnet.eval()

# Define the captioning model (LSTM-based)
class CaptioningModel(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size):
        super(CaptioningModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTMCell(embed_size, hidden_size)
        self.linear = nn.Linear(hidden_size, vocab_size)

    def forward(self, features, captions):
        # Initialize hidden state and cell state
        h, c = torch.zeros_like(features), torch.zeros_like(features)

        # Forward pass through the LSTM for each time step
        outputs = []
        for timestep in range(captions.size(1)):
            embeddings = self.embedding(captions[:, timestep])
            h, c = self.lstm(embeddings, (h, c))
            outputs.append(self.linear(h))

        return torch.stack(outputs, dim=1)

# Define a function to preprocess an image for the ResNet model
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)
    return image

# Load vocabulary (replace with your own vocabulary)
with open('vocab.json', 'r') as f:
    vocab = json.load(f)
    
# Define model hyperparameters
embed_size = 256
hidden_size = 512
vocab_size = len(vocab)

# Initialize captioning model
captioning_model = CaptioningModel(embed_size, hidden_size, vocab_size)

# Load pre-trained weights for the captioning model (optional)
# captioning_model.load_state_dict(torch.load('path/to/your/pretrained/model.pth'))

# Set model to evaluation mode
captioning_model.eval()

# Example usage
image_path = r'C:\Users\SarayuPalem\Downloads\caption1.jpg'  # Replace with the path to your image
image = preprocess_image(image_path)

# Extract features using ResNet
with torch.no_grad():
    features = resnet(image).squeeze()
    
# Generate captions
start_token = torch.tensor([vocab['<start>']])
start_token = start_token.unsqueeze(0).expand(features.size(0), -1).to(device)  # Repeat for the batch size
captions = captioning_model(features, start_token)

# Convert captions to text
caption_text = []
for caption in captions[0]:
    word_idx = torch.argmax(caption).item()
    word = [k for k, v in vocab.items() if v == word_idx][0]
    if word != '<end>':
        caption_text.append(word)
    else:
        break

caption_text = ' '.join(caption_text)
print("Generated Caption:", caption_text)


C:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
C:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to C:\Users\SarayuPalem/.cache\torch\hub\checkpoints\resnet152-394f9c45.pth


FileNotFoundError: [Errno 2] No such file or directory: 'vocab.json'

In [1]:
!pip install transformers

  Using cached https://files.pythonhosted.org/packages/5b/0b/e45d26ccd28568013523e04f325432ea88a442b4e3020b757cf4361f0120/transformers-4.30.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/92/cc/d655b0a1d77dd3e69d1a4f354a868e5b96729cd80ee4912a6ea6fb9e2e64/safetensors-0.4.0-cp37-none-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/ec/1a/610693ac4ee14fcdf2d9bf3c493370e4f2ef7ae2e19217d7a237ff42367d/packaging-23.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/7f/c4/adcbe9a696c135578cabcbdd7331332daad4d49b7c43688bc2d36b3a47d2/huggingface_hub-0.16.4-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/90/06/1f3a3a6fb57bf3e72f63cbf0ae0991540065dd6a13393b89761b38634cb0/tokenizers-0.13.3-cp37-cp37m-win_amd64.whl
  Found existing installation: packaging 19.2
    Uninstalling packaging-19.2:


ERROR: huggingface-hub 0.16.4 has requirement tqdm>=4.42.1, but you'll have tqdm 4.36.1 which is incompatible.
ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\packaging-19.2.dist-info\\INSTALLER'
Consider using the `--user` option or check the permissions.



In [2]:
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer
import torch
from PIL import Image

ModuleNotFoundError: No module named 'transformers'

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

def detect_faces(img_path):
    # Load the cascade
    cascade_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
    face_cascade = cv2.CascadeClassifier(cascade_path)

    if face_cascade.empty():
        print("Failed to load cascade classifier!")
        return

    # Read the input image
    img = cv2.imread(img_path)

    if img is None:
        print("Failed to read image!")
        return

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    # Draw rectangle around the faces
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
# Display the output in Google Colab
    cv2_imshow(img)

# Test
detect_faces('th.jpeg')

In [1]:
!pip install torch torchvision


In [6]:
import torch
import torchvision.transforms as transforms
from torchvision import models, transforms
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

# Load pre-trained ResNet model with weights from ImageNet
resnet = models.resnet152(weights='imagenet')
resnet.eval()

# Preprocessing for images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load pre-trained ResNet model
resnet = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
resnet.eval()

# Replace this with your model for generating captions (LSTM-based, Transformer, etc.)
class CaptioningModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size):
        super(CaptioningModel, self).__init__()
        # Define your captioning model here

    def forward(self, features, captions, lengths):
        # Implement the forward pass for caption generation
        pass

# Example usage
image_path = 'caption1.jpg'  # Replace with the path to your image
image = Image.open(image_path).convert('RGB')
image = transform(image).unsqueeze(0)

# Extract features using ResNet
with torch.no_grad():
    features = resnet(image)
features = features.unsqueeze(1)  # Reshape to match the expected input for the captioning model

# Initialize the captioning model
vocab_size = 10000  # Replace with the actual vocabulary size
embed_size = 256
hidden_size = 512
captioning_model = CaptioningModel(vocab_size, embed_size, hidden_size)

# Generate captions using the captioning model
captions = captioning_model(features, captions=None, lengths=None)
print("Generated Caption:", captions)


ValueError: Unknown value 'imagenet' for ResNet152_Weights.